In [ ]:
!pip install gensim jieba

In [1]:
import re
from gensim.corpora import Dictionary
from gensim.models import TfidfModel, OkapiBM25Model
from gensim.similarities import SparseMatrixSimilarity
import jieba
import heapq

In [ ]:
# 分詞設定
# jieba.load_userdict("userdict.txt")

In [2]:
# 一堆歌詞
li_lyrics = [
'''
作詞：姚若龍     作曲：宇恆　



＊ 前奏 ＊


在東京鐵塔 第一次眺望
看燈火模仿 墜落的星光
我終於到達 但卻更悲傷
一個人完成 我們的夢想

你總說 時間還很多 你可以等我
以前我不懂得 未必明天 就有以後

想念是會呼吸的痛 它活在我身上所有角落
哼你愛的歌會痛 看你的信會痛 連沈默也痛

遺憾是會呼吸的痛 它流在血液中來回滾動
後悔不貼心會痛 恨不懂你會痛 想見不能見最痛

間奏

沒看你臉上 張揚過哀傷
那是種多麼 寂寞的倔強
你拆了城牆 讓我去流浪
在原地等我 把自己綑綁

你沒說 你也會軟弱 需要倚賴我
我就裝不曉得 自由移動 自我地過

想念是會呼吸的痛 它活在我身上所有角落
哼你愛的歌會痛 看你的信會痛 連沈默也痛

遺憾是會呼吸的痛 它流在血液中來回滾動
後悔不貼心會痛 恨不懂你會痛 想見不能見最痛

我發誓不再說謊了 多愛你就會抱你多緊的
我的微笑都假了 靈魂像飄浮著 你在就好了

我發誓不讓你等候 陪你做想做的無論什麼
我越來越像貝殼 怕心被人觸碰 你回來那就好了

能重來那就好了
''',
'''
作詞：李焯雄     作曲：曹宣賓


這一刻 突然覺得好熟悉
像昨天 今天同時在放映
我這句語氣 原來好像你
不就是我們愛過的證據
差一點 騙了自己騙了你
愛與被愛不一定成正比
我知道被疼是一種運氣
但我無法完全交出自己

努力為你改變
卻變不了 預留的伏線
以為在你身邊 那也算永遠
彷彿還是昨天
可是昨天 已非常遙遠
但閉上我雙眼 我還看得見

可惜不是你 陪我到最後
曾一起走 卻走失那路口
感謝那是你 牽過我的手
還能感受那溫柔

那一段 我們曾心貼著心
我想我更有權力關心你
可能你 已走進別人風景
多希望 也有 星光的投影

努力為你改變
卻變不了 預留的伏線
以為在你身邊 那也算永遠
彷彿還是昨天
可是昨天 已非常遙遠
但閉上我雙眼 我還看得見

可惜不是你 陪我到最後
曾一起走 卻走失那路口
感謝那是你 牽過我的手
還能感受那溫柔

可惜不是你 陪我到最後
曾一起走 卻走失那路口
感謝那是你 牽過我的手
還能感受那溫柔

感謝那是你 牽過我的手
還能溫暖我胸口
''',
'''
作詞：易齊     作曲：郭文賢



現在也只能欣賞 唯一的合照一張
淡忘了的是那個街角 想念的是當時的微笑

生活中交錯失望 越想念就越孤單
若再被寂寞迎頭趕上 多感傷原來只是正常

你是不是也在品嚐 一個人的咖啡和天光
是不是也忽然察覺到 多出時間看天色的變換

如果有一天 我們再見面 時間會不會倒退一點
也許我們都忽略 互相傷害之外的感覺
如果哪一天 我們都發現 好聚好散不過是種遮掩
如果我們沒發現 就給彼此多一點時間

生活中交錯失望 越想念就越孤單
若再被寂寞迎頭趕上 多感傷原來只是正常

你是不是也在品嚐 一個人的咖啡和天光
是不是也忽然察覺到 多出時間看天色的變換

如果有一天 我們再見面 時間會不會倒退一點
也許我們都忽略 互相傷害之外的感覺
如果哪一天 我們都發現 好聚好散不過是種遮掩
如果我們沒發現 就給彼此多一點時間

如果有一天 我們再見面 時間會不會倒退一點
也許我們都忽略 互相傷害之外的感覺
如果哪一天 我們都發現 好聚好散不過是種遮掩
如果我們沒發現 就給彼此多一點 時間
''',
'''
作詞：姚若龍     作曲：郭文賢


*:._.:*~*:._.:*~*:._.:*~*:._.:*~*:._.:*~*:._.:*~*:._.:*~*:._.:*~
我無法幫妳預言 委曲求全有沒有用
可是我多麼不捨 朋友愛的那麼苦痛
愛可以不問對錯 至少要喜悅感動
如果他總為別人撐傘
妳何苦非為他等在雨中
泡咖啡讓妳暖手 想擋擋妳心口裡的風
妳卻想上街走走 吹吹冷風會清醒的多
妳說妳不怕分手 只有點遺憾難過
情人節就要來了 剩自己一個
其實愛對了人 情人節每天都過

分手快樂 祝妳快樂 妳可以找到更好的
不想過冬 厭倦沉重 就飛去熱帶的島嶼游泳
分手快樂 請妳快樂 揮別錯的才能和對的相逢
離開舊愛 像坐慢車 看透徹了心就會是晴朗的
沒人能把誰的幸福沒收 妳發誓妳會活的有笑容
*:._.:*~*:._.:*~*:._.:*~*:._.:*~*:._.:*~*:._.:*~*:._.:*~*:._.:*~
*:._.:*~*:._.:*~*:._.:*~*:._.:*~*:._.:*~*:._.:*~*:._.:*~*:._.:*~
泡咖啡讓妳暖手 想擋擋妳心口裡的風
妳卻想上街走走 吹吹冷風會清醒的多
妳說妳不怕分手 只有點遺憾難過
情人節就要來了 剩自己一個
其實愛對了人 情人節每天都過
分手快樂 祝妳快樂 妳可以找到更好的
不想過冬 厭倦沉重 就飛去熱帶的島嶼游泳
分手快樂 請妳快樂 揮別錯的才能和對的相逢
離開舊愛 像坐慢車 看透徹了心就會是晴朗的
沒人能把誰的幸福沒收 妳發誓妳會活的有笑容
妳自信時候真的美多了
*:._.:*~*:._.:*~*:._.:*~*:._.:*~*:._.:*~*:._.:*~*:._.:*~*:._.:*~
●○●○●○●○●○●○●Ｅｎｄ○●○●○●○●○●○●○●
''',
'''
作詞：Lee/Jae-Kyung     作曲：Lee/Sang-Ho



█ ☆█ ◢◤◥◣ 前 ███◣ ███◣ █ ☆█
█▅▅█ █▄▄█ █ ☆█ █ ☆█ ◥███
█ █ █ █ 奏 ███◤ ███◤ █
█ █ █ ☆█ █ ☆ █ ☆ ◥██◤

愛你等於擁有一片天空 任何風吹草動
都有你存在其中 自然而然的輕鬆

一路到夏天的尾聲 無所謂到過於激動
我們有笑容 我們曾心動 不再是無動於衷

無條件為你不顧明天的安穩
為你變堅強相信你的眼神
不敢想不敢問 有一天壞的可能
無條件為你放棄單獨的旅程
為你堅強就不怕犧牲
我的靈魂 如此沸騰 為我愛的人

＊＊＊＊間 奏＊＊＊＊

喜歡複雜還是習慣單純 我願盡力完成
你在我心中幾分 難以形容的責任

一路到夏天的尾聲 無所謂到過於激動
我們有笑容 我們曾心動 不再是無動於衷

無條件為你不顧明天的安穩
為你變堅強相信你的眼神
不敢想不敢問 有一天壞的可能
無條件為你放棄單獨的旅程
為你堅強就不怕犧牲
我的靈魂 如此沸騰 為我愛的人

愛一個人付出才會完整
無條件 越愛就越深 永遠不分 啊 .......

無條件為你不顧明天的安穩
為你變堅強相信你的眼神
不敢想不敢問 有一天壞的可能
無條件為你放棄單獨的旅程
為你堅強就不怕犧牲
我的靈魂 如此沸騰 為我愛的人

╭╮ ╭╮
～‧‧╭┴┴─────┴┴╮
～‧‧│ │＼｜／
～‧‧│ ● ● │─☆─
～‧‧│○ ╰┬┬┬╯ ○│／｜＼
～☆‧│ ╰─╯ ／
～‧‧╰─┬○────┬○╯
～‧‧‧╭│ │╮
～～‧‧╰┴─────┴╯
''',
'''
[梁靜茹 “勇氣” 歌詞]

[主歌]
終於做了這個決定
別人怎麼說我不理
只要你也一樣的肯定
我願意天涯海角都隨你去
我知道一切不容易
我的心一直溫習說服自己
最怕你忽然說要放棄

[副歌]
愛真的需要勇氣
來面對流言蜚語
只要你一個眼神肯定
我的愛就有意義
我們都需要勇氣
去相信會在一起
人潮擁擠我能感覺你
放在我手心裡
你的真心

[主歌]
終於做了這個決定
別人怎麼說我不理
只要你也一樣的肯定
我願意天涯海角都隨你去
我知道一切不容易
我的心一直溫習說服自己 (ooh, ooh)
最怕你忽然說要放棄 (ah-hah)
See upcoming pop shows
Get tickets for your favorite artists
You might also like
JENNIE - Mantra (中文翻译 - Simplified)
Genius Chinese Translations (中文翻譯/中文翻译)
Thick Of It
KSI
沉船仔(Lovestruck)
Tr33 (HK)
[副歌]
愛真的需要勇氣 (ooh-ooh)
來面對流言蜚語 (ooh-ooh)
只要你一個眼神肯定 (ah-ah)
我的愛就有意義
我們都需要勇氣 (ooh-ooh)
去相信會在一起 (ooh-ooh)
人潮擁擠我能感覺你 (ah-ah)
放在我手心裡
你的真心

[間奏]
如果我的堅強任性 (ah-ah)
會不小心傷害了你
你能不能溫柔提醒 (ah-ah)
我雖然心太急
更害怕錯過你

[副歌]
愛真的需要勇氣 (ooh-ooh)
來面對流言蜚語 (ah-ah)
只要你一個眼神肯定 (ooh-ooh)
我的愛就有意義
我們都需要勇氣 (ooh-ooh)
去相信會在一起 (ooh-ooh, ah-ah)
人潮擁擠我能感覺你
放在我手心裡
你的真心
''',
]

In [3]:
# 將文件 (歌詞) 斷詞
li_seq = []
for seq in li_lyrics:
    seq = re.sub(r"\s| |\n", "", seq) # 簡單清理
    li_seq.append(list(jieba.cut(seq))) # 或者用 jieba.lcut(seq)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\darren\AppData\Local\Temp\jieba.cache
Loading model cost 0.326 seconds.
Prefix dict has been built succesfully.


In [4]:
# 用 BM25 模型計算相似度(相關性)

'''
定義函式 bm25_similarity：
- list_seq 是一個文件的列表，每個文件都是一個詞的列表；
- query 是查詢字符串。
'''
def bm25_similarity(list_seq, query):
    '''
    建立詞典：
    - 使用 list_seq 中的所有詞彙建立一個 Dictionary 物件，為每個詞彙分配一個唯一的ID。
    '''
    dictionary = Dictionary(list_seq)

    '''
    初始化 BM25 模型：
    - 使用先前建立的詞典，來初始化一個 OkapiBM25Model 物件，這是用於資訊檢索 (Information Retrieval) 的統計模型。
    '''
    bm25_model = OkapiBM25Model(dictionary=dictionary)

    '''
    建立 BM25 語料庫：
    - dictionary.doc2bow：將每個文件（詞的列表）轉換為詞袋（Bag-of-Words）表示，即每個詞的ID及其在文件中的出現次數。
    - map 函式：對 list_seq 中的每個文件應用 doc2bow 轉換。
    - bm25_model[...]：對轉換後的語料庫應用 BM25 權重，得到加權的語料庫 bm25_corpus。
    '''
    bm25_corpus = bm25_model[list(map(dictionary.doc2bow, list_seq))]

    '''
    SparseMatrixSimilarity：創建一個稀疏矩陣相似度對象，用於快速計算查詢與文件之間的相似度。
    - bm25_corpus：BM25 加權的語料庫。
    - num_docs：文件數量。
    - num_terms：詞典中詞的數量。
    - normalize_queries=False：不對查詢進行 normalization。
    - normalize_documents=False：不對文件進行 normalization。
    '''
    bm25_index = SparseMatrixSimilarity(
        bm25_corpus, 
        num_docs=len(list_seq), 
        num_terms=len(dictionary), 
        normalize_queries=False, 
        normalize_documents=False
    )

    '''
    對查詢進行分詞：
    - 使用 jieba.lcut 對查詢字符串進行分詞。
    '''
    query = jieba.lcut(query)

    '''
    建立 TF-IDF 模型：
    - 參數 smartirs='bnn'：指定權重計算的方式，其中 'bnn' 表示二值化的詞頻和未正則化的逆向文件頻率 (IDF)。
    '''
    tfidf_model = TfidfModel(
        dictionary=dictionary, 
        smartirs='bnn' # default: nfc
    )

    '''
    轉換查詢向量：
    - dictionary.doc2bow(query)：將查詢詞列表轉換為詞袋表示。
    - tfidf_model[...]：對詞袋表示的查詢應用 TF-IDF 權重，得到加權的查詢向量 tfidf_query。
    '''
    tfidf_query = tfidf_model[dictionary.doc2bow(query)]

    '''
    計算相似度：
    - 使用之前建立的 bm25_index，計算查詢向量與語料庫中每個文件的相似度。
    '''
    similarities = bm25_index[tfidf_query]
    
    return similarities

In [5]:
# 顯示前 top_k 高相似度的文章
def get_page_score(similarities, top_k):
    '''
    (文章索引, 查詢後的相似分數)
    [(0, 0.0), (1, 0.0), (2, 0.0), (3, 3.4350295), (4, 2.4153714), (5, 0.0), (6, 0.0), ...]
    '''
    similarities = list(enumerate(similarities))

    '''
    前 top_k 高相似度的文章
    [(176, 15.524609), (177, 9.0146), (116, 7.6217546), (100, 6.096363), (159, 4.855205)]
    '''
    top_items  = heapq.nlargest(
        top_k, 
        similarities,
        key=lambda x: x[1]
    )

    return [(doc_index, score) for doc_index, score in top_items]

In [6]:
# 基本設定
top_k = 3

In [7]:
# 要查詢的字串
query = '''愛真的需要勇氣'''

# 查詢字串被斷詞後的樣子
print(jieba.lcut(query))

# 計算相似度
sim = bm25_similarity(li_seq, query)

# 顯示前 top_k 高相似度的文章
for doc_id, score in get_page_score(sim, top_k):
    print(f'Document Num: {doc_id}, Score: {score:.4f}')

['愛', '真的', '需要', '勇氣']
Document Num: 5, Score: 5.2933
Document Num: 0, Score: 1.0498
Document Num: 3, Score: 0.9200
